## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [7]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data.shape, data

((60, 4, 1000),
 array([[[-10.935548  ,   3.9000723 ,  -9.886959  , ...,  11.973424  ,
           12.83266   , -21.400776  ],
         [ -5.68035   ,   0.72187126,  10.169985  , ...,   6.2162094 ,
           -9.350651  ,  -6.3647695 ],
         [ 10.466743  , -23.645386  ,  15.096241  , ...,  12.950799  ,
           13.546511  ,  11.009279  ],
         [  3.3458476 ,  -4.0723443 ,  16.848394  , ...,   7.5587535 ,
          -11.184831  , -23.363348  ]],
 
        [[ -6.3337755 ,  -7.6228714 ,  18.276823  , ...,   5.344354  ,
          -14.598693  ,   6.400412  ],
         [-11.915309  ,   5.650533  ,   3.142348  , ...,  -0.7373454 ,
          -20.239872  ,   6.2214127 ],
         [ -2.0500288 ,   0.80564815,   3.1092582 , ...,  -4.3498645 ,
           -7.9891024 ,  -7.5919075 ],
         [ -4.9574685 ,  -7.567727  ,  12.659855  , ...,  -6.941405  ,
            3.987537  ,  -6.6128716 ]],
 
        [[  3.2462327 ,   9.585448  ,  -0.24457598, ...,  12.908168  ,
            0.09098721,  -8

In [8]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[1] - segment) / step) + 1  # Adjusted for the second dimension
ids = np.arange(n_win) * step

# janelas do dado no domínio do tempo
chunks_time = np.array([data[:, k:(k + segment), :].transpose(0, 2, 1) for k in ids])

# janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=segment, noverlap=step-1)  # Adjusted noverlap
chunks_freq = chunks_freq.transpose(0, 2, 3, 1)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Domínio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Domínio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Domínio do tempo: (0,) - (classes+ensaios, canais, janelas, linhas)
Domínio da frequência:  (60, 33, 29, 4) - (classes+ensaios, canais, janelas, linhas)


*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).